In [4]:
import mlflow

In [ ]:
!pip3 install -r mlruns/0/f99975bdab2748d9a5fca53513d49bac/artifacts/ai_text_classifier/requirements.txt

### Random story generated by perplexity

In [5]:
ai_test = '''Sarah stared at her monitor, her eyes burning from the fourteen straight hours of debugging. The production server was down, and somewhere in her code lurked a bug that was costing the company thousands by the hour. Coffee cups littered her desk like fallen soldiers, testament to her ongoing battle with the elusive error.

"It has to be in the authentication module," she muttered, scrolling through countless lines of code. The error logs showed users getting kicked out randomly, but the stack trace led nowhere. It was like chasing a ghost through a maze of semicolons and brackets.

Her colleague Mark had left hours ago, patting her shoulder with a sympathetic "You'll find it." The office was empty now, the usual buzz of keyboards replaced by the hum of servers and the soft whir of her laptop fan.

She ran the test suite for the hundredth time, watching the green dots appear one by one. Everything passed. Yet in production, the system was failing. Sarah pulled up the git history, reviewing each commit from the past week.

That's when she saw it.

It wasn't a bug at all. In the config file, pushed three days ago, someone had changed the session timeout from hours to milliseconds. The system wasn't broken – it was working exactly as instructed, just with the wrong unit of time.

Sarah let out a laugh that echoed through the empty office. One tiny decimal point in the wrong place, and here she'd been hunting for a complex bug in her authentication logic.

She made the fix, pushed the code, and watched as the system came back to life. Users started flowing back in, their sessions now lasting the intended duration instead of mere milliseconds.

As she packed up her laptop, Sarah couldn't help but smile. Programming had taught her many lessons, but perhaps the most important was that the simplest explanation was often the correct one. Sometimes the most daunting bugs weren't bugs at all, just human oversight in its purest form.

She sent a quick message to the team: "Fixed. Config issue. Going home to sleep for a year."

As she walked to her car, the first rays of dawn were breaking over the horizon. Another crisis solved, another lesson learned, another story to tell at the next team meeting. Such was the life of a programmer – where the smallest details could create the biggest problems, and where victory often meant finding nothing wrong with your code at all.
'''

### Excerpt from "The Dead" by James Joyce (1914)

In [6]:
human_test = '''Gabriel, leaning on his elbow, looked for a few moments unresentfully on her tangled hair and half-open mouth, listening to her deep-drawn breath. So she had had that romance in her life: a man had died for her sake. It hardly pained him now to think how poor a part he, her husband, had played in her life. He watched her while she slept, as though he and she had never lived together as man and wife. His curious eyes rested long upon her face and on her hair: and, as he thought of what she must have been then, in that time of her first girlish beauty, a strange, friendly pity for her entered his soul. He did not like to say even to himself that her face was no longer beautiful, but he knew that it was no longer the face for which Michael Furey had braved death.

Perhaps she had not told him all the story. His eyes moved to the chair over which she had thrown some of her clothes. A petticoat string dangled to the floor. One boot stood upright, its limp upper fallen down: the fellow of it lay upon its side. He wondered at his riot of emotions of an hour before. From what had it proceeded? From his aunt’s supper, from his own foolish speech, from the wine and dancing, the merry-making when saying good-night in the hall, the pleasure of the walk along the river in the snow. Poor Aunt Julia! She, too, would soon be a shade with the shade of Patrick Morkan and his horse. He had caught that haggard look upon her face for a moment when she was singing Arrayed for the Bridal. Soon, perhaps, he would be sitting in that same drawing-room, dressed in black, his silk hat on his knees. The blinds would be drawn down and Aunt Kate would be sitting beside him, crying and blowing her nose and telling him how Julia had died. He would cast about in his mind for some words that might console her, and would find only lame and useless ones. Yes, yes: that would happen very soon.

The air of the room chilled his shoulders. He stretched himself cautiously along under the sheets and lay down beside his wife. One by one, they were all becoming shades. Better pass boldly into that other world, in the full glory of some passion, than fade and wither dismally with age. He thought of how she who lay beside him had locked in her heart for so many years that image of her lover’s eyes when he had told her that he did not wish to live.'''

In [3]:
def predict_with_mlflow(text, model_uri="models:/ai_text_classifier/latest"):
    try:
        # Load pipeline
        loaded_pipeline = mlflow.sklearn.load_model(model_uri)
        
        # Make prediction
        prediction = loaded_pipeline.predict([text])
        probabilities = loaded_pipeline.predict_proba([text])
        
        print(f"Prediction: {prediction[0]}")
        print(f"Probabilities: {probabilities[0]}")
        
        return prediction[0]
    except Exception as e:
        print(f"Error during prediction: {str(e)}")
        return None

In [7]:
pred_0 = predict_with_mlflow(human_test)
pred_1 = predict_with_mlflow(ai_test)

/Users/joshlongenecker/Development/random/.venv/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Prediction: 0
Probabilities: [0.8204559  0.17954409]
Prediction: 1
Probabilities: [0.01657999 0.98342   ]


## Test

In [1]:
from datasets import load_dataset

ds = load_dataset("andythetechnerd03/AI-human-text")
df_test = ds['test'].to_pandas()

In [ ]:
# Run predictions and add prediction column and get confusion matrix to show model accuracy
from sklearn.metrics import confusion_matrix
import pandas as pd

# Load model once instead of for each prediction
loaded_pipeline = mlflow.sklearn.load_model("models:/ai_text_classifier/latest")

#get a sample of 1k of each label from the ds dataframe
df_0 = df_test[df_test['generated'] == 0].sample(n=200, random_state=40)
df_1 = df_test[df_test['generated'] == 1].sample(n=200, random_state=40)
df_sample = pd.concat([df_0, df_1]).reset_index(drop=True)

predictions = loaded_pipeline.predict(df_sample.text.tolist())
df_sample['prediction'] = predictions

In [9]:
from sklearn.metrics import classification_report
print(classification_report(df_sample.generated, df_sample.prediction))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86       200
           1       0.90      0.79      0.84       200

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

